In [8]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout
from weaviate.connect import ConnectionParams


HTTP_HOST = "localhost"
HTTP_PORT = 8080
HTTP_SECURE = False

GRPC_HOST = "localhost"
GRPC_PORT = 50051
GRPC_SECURE = False

INTIALISATION_TIMEOUT_S = 2
QUERY_TIMEOUT_S = 45
INSERT_TIMEOUT_S = 120

COLLECTION_NAME = "Cap3DMM"
DATA_UPLOAD_COLLECTION_NAME = "UploadCap3DMM"


with weaviate.WeaviateClient(
    connection_params=ConnectionParams.from_params(
        http_host=HTTP_HOST,
        http_port=HTTP_PORT,
        http_secure=HTTP_SECURE,
        grpc_host=GRPC_HOST,
        grpc_port=GRPC_PORT,
        grpc_secure=GRPC_SECURE,
    ),
    additional_config=AdditionalConfig(
        timeout=Timeout(
            init=INTIALISATION_TIMEOUT_S, query=QUERY_TIMEOUT_S, insert=INSERT_TIMEOUT_S
        ),  # Values in seconds
    ),
) as client:
    assert (
        client.is_live()
    ), "Weaviate client is not live"  # This will raise an exception if the client is not live
    print("Client connection established")

    cap3d = client.collections.get(COLLECTION_NAME)
    cap3d_object_count = cap3d.aggregate.over_all(total_count=True).total_count
    cap3d_upload = client.collections.get(DATA_UPLOAD_COLLECTION_NAME)
    cap3d_upload_image_count = cap3d_upload.aggregate.over_all(
        total_count=True
    ).total_count

    print(f"Number of objects in {COLLECTION_NAME} collection: {cap3d_object_count}")
    print(
        f"Number of objects in {DATA_UPLOAD_COLLECTION_NAME} collection: {cap3d_upload_image_count}"
    )

    uuids = [
        "46c932c1-d44e-5b30-95d7-4c8909d02fb8",
        "91faaa0e-6a0f-5587-a2dd-fd6e2f3ca3e8",
        "db04848b-c5dd-5ec1-a340-adcc209e9b5e",
        "e574ddc1-3171-5a81-a3aa-d4e2928b9f54",
        "baa41176-79e8-5fbb-8b16-7444f228f88a",
    ]
    obj = cap3d.query.fetch_object_by_id(uuids[0], include_vector=True)

Client connection established
Number of objects in Cap3DMM collection: 0
Number of objects in UploadCap3DMM collection: 780


In [50]:
1000 * 20

20000

In [51]:
5280 / 20

264.0

In [20]:
obj

In [42]:
obj.vector

{}

In [43]:
from pathlib import Path

PATH_TO_EXAMPLE_OBJECTS = "/home/yunusskeete/Documents/data/3D/Cap3D/local-split/unzips/compressed_imgs_perobj_00.zip/Cap3D_Objaverse_renderimgs"
path_to_example_objects = Path(PATH_TO_EXAMPLE_OBJECTS)
sum(1 for _ in path_to_example_objects.iterdir())

15000